FIAP - IA & Machine Learning

# Aula 09 - Regressão Linear

Vamos passar pelo pipeline de ciência de dados, implementando os passos em python e realizando uma Regressão Linear como modelo preditivo.

## Pipeline de ciência de dados

Relembrando as etapas do processo de ciência de dados:

![image.png](https://www.sudeep.co/images/post_images/2018-02-09-Understanding-the-Data-Science-Lifecycle/chart.png)

### Entendimento do problema

É possível prever a altura dos filhos baseado nas alturas dos pais?

### Obtenção dos dados

Este conjunto de dados lista as observações individuais de 934 crianças em 205 famílias nas quais Galton (1886) baseou sua tabulação cruzada, mostrando a relação entre as alturas dos pais e de seus filhos.

Dados baixados [deste link](https://raw.githubusercontent.com/data-8/materials-fa17/master/lec/galton.csv)

Dicionário de dados retirado [daqui](https://vincentarelbundock.github.io/Rdatasets/doc/HistData/GaltonFamilies.html)

Os dados são:

- `family`: ID de família, um fator com níveis 001-204
- `father`: altura do pai
- `mother`: altura da mãe
- `midparentHeight`: altura parental média, calculada como (pai + 1,08 \* mãe) / 2
- `children`: número de filhos nesta família
- `childNum`: número desta criança dentro da família. As crianças são listadas em ordem decrescente de altura para meninos, seguidos por meninas
- `gender`: gênero infantil, fator com níveis feminino e masculino
- `childHeight`: altura da criança

**OBSERVAÇÃO**: os dados das alturas estão em `inches`. Para transformar em `cm`, podemos usar a fórmula: "1 inch =
2.54 centimeters"


In [316]:
# Imports necessários
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [317]:
df = pd.read_csv("data/galton.csv")
df.head(5)

,family,father,mother,midparentHeight,children,childNum,gender,childHeight
0,1,78.5,67.0,75.43,4,1,male,73.2
1,1,78.5,67.0,75.43,4,2,female,69.2
2,1,78.5,67.0,75.43,4,3,female,69.0
3,1,78.5,67.0,75.43,4,4,female,69.0
4,2,75.5,66.5,73.66,4,1,male,73.5


In [318]:
# Quantos dados?
df.shape

(934, 8)

In [319]:
# Quais os tipos? Dados faltantes?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   family           934 non-null    object 
 1   father           934 non-null    float64
 2   mother           934 non-null    float64
 3   midparentHeight  934 non-null    float64
 4   children         934 non-null    int64  
 5   childNum         934 non-null    int64  
 6   gender           934 non-null    object 
 7   childHeight      934 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 58.5+ KB


In [320]:
# Quais são os valores da coluna família? Notem que tem elementos com "A", por isso o Pandas
# converteu tudo para string (object), pois ao forçar conversão para float ia dar problema
df["family"].values

array(['1', '1', '1', '1', '2', '2', '2', '2', '3', '3', '4', '4', '4',
       '4', '4', '5', '5', '5', '5', '5', '5', '6', '7', '7', '7', '7',
       '7', '7', '8', '8', '8', '9', '10', '11', '11', '11', '11', '11',
       '11', '11', '11', '12', '13', '13', '14', '14', '15', '15', '15',
       '16', '16', '16', '16', '16', '16', '16', '16', '16', '17', '17',
       '17', '17', '17', '17', '18', '18', '18', '19', '20', '20', '20',
       '20', '20', '20', '20', '20', '21', '21', '21', '22', '22', '22',
       '23', '23', '23', '23', '23', '23', '23', '24', '25', '25', '26',
       '26', '26', '26', '26', '27', '27', '27', '28', '28', '28', '28',
       '28', '28', '29', '29', '29', '30', '31', '31', '31', '31', '31',
       '31', '32', '32', '32', '32', '32', '33', '33', '33', '33', '33',
       '34', '35', '35', '35', '35', '35', '36', '36', '36', '36', '37',
       '37', '37', '37', '38', '38', '38', '38', '38', '38', '39', '39',
       '40', '40', '40', '40', '40', '41', '42', '42'

In [321]:
# Filtros: todas os filhos homens
df[df["gender"] == "male"]

,family,father,mother,midparentHeight,children,childNum,gender,childHeight
0,1,78.5,67.0,75.43,4,1,male,73.2
4,2,75.5,66.5,73.66,4,1,male,73.5
5,2,75.5,66.5,73.66,4,2,male,72.5
8,3,75.0,64.0,72.06,2,1,male,71.0
10,4,75.0,64.0,72.06,5,1,male,70.5
...,...,...,...,...,...,...,...,...
918,199,64.0,64.0,66.56,7,2,male,68.0
924,200,64.0,63.0,66.02,1,1,male,64.5
925,201,64.0,60.0,64.40,2,1,male,66.0
929,203,62.0,66.0,66.64,3,1,male,64.0


In [322]:
# Existem quantos valores únicos para cada coluna?
df.nunique()

family             205
father              35
mother              29
midparentHeight    140
children            12
childNum            15
gender               2
childHeight         67
dtype: int64

In [323]:
# Filtros múltiplos: coloque cada condição entre parênteses! Use:
#    & : para AND (and)
#    | : para OR (ou)
# Filtrando todas as meninas com altura maior que 70
df[(df["childHeight"] > 70) & (df["gender"] == "female")]

,family,father,mother,midparentHeight,children,childNum,gender,childHeight
26,7,74.0,68.0,73.72,6,5,female,70.5
28,8,74.0,66.5,72.91,3,1,female,70.5
102,28,72.0,63.0,70.02,6,3,female,70.5


In [324]:
# Alguma info estatística útil?
df.describe()

,father,mother,midparentHeight,children,childNum,childHeight
count,934.000000,934.000000,934.000000,934.000000,934.000000,934.000000
mean,69.197109,64.089293,69.206773,6.171306,3.585653,66.745931
std,2.476479,2.290886,1.802370,2.729025,2.361410,3.579251
min,62.000000,58.000000,64.400000,1.000000,1.000000,56.000000
25%,68.000000,63.000000,68.140000,4.000000,2.000000,64.000000
50%,69.000000,64.000000,69.248000,6.000000,3.000000,66.500000
75%,71.000000,65.875000,70.140000,8.000000,5.000000,69.700000
max,78.500000,70.500000,75.430000,15.000000,15.000000,79.000000


### Limpeza dos dados


In [325]:
# Removendo duplicatas
df.drop_duplicates(inplace=True)

In [326]:
# Removendo dados faltantes
df.dropna(inplace=True)

In [327]:
# Tamanho final: aqui é o mesmo tamanho da entrada, porque não há duplicadas ou valores faltantes
df.shape

(934, 8)

Vamos trabalhar com dados em **metros**? Não somos obrigados.


In [328]:
# Funcao
def inch2m(inch):
    return inch * 2.54 / 100

In [329]:
inch2m(79)

2.0066

In [330]:
# Converte a coluna das alturas do PAI
# Para cada linha da coluna FATHER aplica a função "inch2m()", salve o valor na própria coluna FATHER
df["father"] = df["father"].apply(inch2m)

In [331]:
df.sample(10)["father"]

933    1.5875
278    1.7780
34     1.8796
719    1.7018
511    1.7526
631    1.7272
825    1.6891
467    1.7653
584    1.7399
270    1.7907
Name: father, dtype: float64

Agora faça o mesmo para as demais colunas de alturas!


In [332]:
colunas = ["mother", "midparentHeight", "childHeight"]

In [333]:
# Para cada valor da lista "colunas", salve o valor na variável "coluna" e
# faça as mesmas alterações que da coluna father
for c in colunas:
    df[c] = df[c].apply(inch2m)

In [334]:
df.sample(5)[colunas]

,mother,midparentHeight,childHeight
699,1.5240,1.686560,1.5240
600,1.6510,1.755140,1.6764
915,1.6256,1.690624,1.5748
427,1.6764,1.781556,1.6637
649,1.6002,1.727708,1.7272


Sabemos que os modelos só trabalham com números, então não podemos deixar a coluna `gender` como está! Precisamos transformá-la!

O código da célula abaixo possui o mesmo efeito que o seguinte código, com uma função explícita:

```python
def sex(x):
    if x == "male":
        return 0
    else:
        return 1

df["gender"] = df["gender"].apply(sex)
```


In [335]:
# Aqui usamos uma função implícita chamada "lambda". Para cada linha, pegue o valor x da coluna
# gender e verifique se é masculino. Se for, retorne o valor 0, do contrário retorne o valor 1
df["gender"] = df["gender"].apply(lambda x: 0 if x == "female" else 1)

In [336]:
df.sample(5)["gender"]

133    1
343    1
629    0
776    1
780    1
Name: gender, dtype: int64

In [337]:
# Removendo colunas que não serão úteis
df1 = df[["father", "mother", "midparentHeight", "gender", "childHeight"]]

df1

,father,mother,midparentHeight,gender,childHeight
0,1.9939,1.7018,1.915922,1,1.85928
1,1.9939,1.7018,1.915922,0,1.75768
2,1.9939,1.7018,1.915922,0,1.75260
3,1.9939,1.7018,1.915922,0,1.75260
4,1.9177,1.6891,1.870964,1,1.86690
...,...,...,...,...,...
929,1.5748,1.6764,1.692656,1,1.62560
930,1.5748,1.6764,1.692656,0,1.57480
931,1.5748,1.6764,1.692656,0,1.54940
932,1.5875,1.6002,1.657858,1,1.68910


### Modelagem


In [338]:
# Separação de DADOS e LABEL
X = df1.drop(columns=["childHeight"])
y = df1["childHeight"]

In [339]:
# Dividindo dados para TREINO e TESTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [340]:
# Treinando o modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [341]:
# Fazendo as predições
y_pred = modelo.predict(X_test)

### Avaliação


In [342]:
# Erro quadrático médio
mean_squared_error(y_test, y_pred)

0.0030882702629144963

In [343]:
# Erro absoluto médio
mean_absolute_error(y_test, y_pred)

0.043607902738668866

In [344]:
# R²
r2_score(y_test, y_pred)

0.6068275920421932

### Interpretando os coeficientes encontrados!

Vamos dar uma olhada nos coeficientes encontrados para cada característica:


In [345]:
# Intercept (parâmetro "b" da reta caracterizada por: y = ax + b)
modelo.intercept_

0.3605113261509383

In [346]:
modelo.coef_

array([-4.00666530e+12, -4.32719853e+12,  8.01333060e+12,  1.35620117e-01])

In [347]:
X_train.columns

Index(['father', 'mother', 'midparentHeight', 'gender'], dtype='object')

In [348]:
# y = a * X + b
# y = a1 * X1 + a2 * X2 + b

In [349]:
X_new = pd.DataFrame(
    columns=["father", "mother", "midparentHeight", "gender"],
    data=[[1.82, 1.55, 1.747, 1]],
)

modelo.predict(X_new)

array([1.76566269])

### Criando um dataframe df3 com os valores do teste e suas predições


In [350]:
# O dataframe df3 recebe o filtro do df2 pelos índices do TESTE, resetando os índices de 0 até seu tamanho
df3 = df1.iloc[X_test.index].reset_index()

In [351]:
# A coluna nova "predição" recebe os valores preditos pelo modelo
df3["prediction"] = y_pred

In [352]:
df3.sample(10)

,index,father,mother,midparentHeight,gender,childHeight,prediction
56,376,1.7780,1.6002,1.753108,0,1.5748,1.629066
84,32,1.8796,1.6637,1.838198,0,1.6637,1.693519
202,35,1.8796,1.5748,1.790192,0,1.7272,1.664222
101,479,1.7526,1.6510,1.767840,1,1.7018,1.769691
250,878,1.6510,1.6510,1.717040,0,1.5748,1.590980
28,205,1.8034,1.6002,1.765808,0,1.6002,1.636879
107,576,1.7399,1.6891,1.782064,1,1.7526,1.778480
27,141,1.8034,1.6764,1.806956,1,1.7526,1.800941
119,145,1.8034,1.6764,1.806956,0,1.6002,1.665199
191,509,1.7526,1.6129,1.747266,0,1.6510,1.625160


In [360]:
df4 = df3[["childHeight", "prediction"]].rename(columns={"childHeight": "label"})

df4.sample(50).apply(lambda x: round(x, 2))

,label,prediction
268,1.80,1.72
119,1.60,1.67
272,1.70,1.62
255,1.73,1.78
224,1.64,1.70
168,1.60,1.59
136,1.79,1.71
82,1.70,1.78
29,1.71,1.75
76,1.66,1.64
